In [ ]:
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import nltk

In [17]:
path = '/kaggle/input/twitter-sentiment-dataset/Twitter_Data.csv'
df=pd.read_csv(path)
df.head()


In [18]:
df.tail()

In [22]:
plt.figure(figsize=(8,10))
sns.countplot(x=df['category']).tail()

In [24]:
df.info()

In [27]:
# removing rows containing null values
df.drop(df[df['clean_text'].isna()].index, inplace=True)
df.drop(df[df['category'].isna()].index, inplace=True)

In [28]:

df.info()

In [29]:

# importing nltk library for text preprocessing
import nltk 
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer, WordNetLemmatizer
import re
nltk.download('wordnet')


In [37]:
lemm = WordNetLemmatizer()

In [38]:
# preprocessing the text
def preprocessing(line):
  tokens = []
  for token in line.split():
      tokens.append(lemm.lemmatize(token))
  return ' '.join(tokens)

In [35]:
df.clean_text = df.clean_text.apply(lambda x: preprocessing(x))

In [39]:
df['clean_text'][0]

In [41]:
df['category'] = [2 if x == -1 else x for x in df['category']]

In [42]:

df.head()

In [43]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

In [46]:
#tokenizer i.e vectorizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)
word_index = tokenizer.word_index#tokenizer i.e vectorizing the text
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.clean_text)
word_index = tokenizer.word_index

In [47]:
vocab_size = len(word_index)+1

In [48]:
#padding the tokenized sequences to same length
from keras.preprocessing.sequence import pad_sequences
MAX_SEQUENCE_LENGTH = 200
lines = pad_sequences(tokenizer.texts_to_sequences(df.clean_text),
                        maxlen = MAX_SEQUENCE_LENGTH)

In [50]:
df.clean_text = lines.tolist()

In [51]:
df.head()

In [53]:
from sklearn.preprocessing import LabelEncoder
label = LabelEncoder()
encodedy = label.fit(y=df['category'])
y = label.transform(df['category'])
from keras.utils import np_utils
dummy_y = np_utils.to_categorical(y)
dummy_y.shape
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df['clean_text'],df['category'],test_size=0.2,random_state=101)

In [55]:
import numpy as np
X_train = np.vstack(X_train.values)
y_train = np.vstack(y_train.values)
X_val = np.vstack(X_test.values)
y_val = np.vstack(y_test.values)

In [56]:

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, LSTM, Dense,Dropout

model = tf.keras.Sequential()

# Input layer
model.add(Input(shape=(None,)))

# Embedding layer
model.add(Embedding(input_dim=vocab_size,output_dim=200,trainable=True))

# LSTM layer
model.add(LSTM(64, activation='relu'))

# Fully connected layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))

# Output layer
model.add(Dense(3, activation='sigmoid'))

model.summary()

In [58]:
model.compile(optimizer='adam', loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), metrics=['accuracy'])
history = model.fit(X_train, y_train,
                    epochs=2, batch_size=512,
                    verbose=1,shuffle=True,validation_data=(X_val,y_val))

In [61]:
predictions = model.predict(X_val)
print(history.history)

In [62]:

print("Evaluate on test data")
results = model.evaluate(X_val, y_val, batch_size=128)
print("test loss, test acc:", results)

In [63]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [64]:
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()